In [1]:
# required for jupyter notebook
%matplotlib inline 

import numpy as np
import pandas as pd

import os
import math

### Mean aesthetic score = 2.455

In [8]:
MEAN_AES_SCORE = 2.455

In [16]:
raw_aestheticScores_df = pd.read_csv(os.path.join('..', 'dataset', 'BanglaLekha-Isolated', 'aesthetic-scores.csv'))

In [17]:
raw_aestheticScores_df.head()

,Form No,Out Of 5 (1st Person),Out of 5 (2nd Person),Out of 5 (3rd Person),Average
0,1,2,4,5,3.666667
1,2,1,3,4,2.666667
2,3,3,5,4,4.000000
3,4,2,3,4,3.000000
4,5,1,3,3,2.333333


In [18]:
'''
returns a map of key->value : form_id->aesthetic_score
'''
def get_form_aesScores(raw_aestheticScores_df):
    
    form_aesScores = {} 
    
    form_ids = []
    aes_scores = []
    for form_id in raw_aestheticScores_df['Form No']:
        form_ids.append(form_id)
    for aes_score in raw_aestheticScores_df['Average']:
        aes_scores.append(aes_score)
        
    for i in range(len(form_ids)):
        form_aesScores[form_ids[i]] = aes_scores[i]
        
    return form_aesScores

In [19]:
mp_formWise_aesScores = get_form_aesScores(raw_aestheticScores_df)
len(mp_formWise_aesScores)

2000

In [23]:
'''
create 4 PARALLEL lists- image_file_name[], aesthetic_score[], aesthetic_quality[], prob_good[]
image_file_name[] = name of the sample image file
aesthetic_score[] = aesthetic score assigned in decimal point number
aesthetic_quality[] = label- 'good' or 'bad'
prob_good[] = 1 if good, 0 if bad 

returns a dataframe with each list as a column
'''
def prepare_hwAesthetics_df(mp_formWise_aesScores, begin=1, end=84, mean_aesScore = 2.455):

    image_file_name, aesthetic_score, aesthetic_quality, prob_good = [], [], [], []
    
    images_dir = os.path.join('..', 'dataset', 'BanglaLekha-Isolated', 'Images')
    
    for i in range(begin, end+1):
        
        curr_dir = os.path.join(images_dir, str(i))
        
        for file_name in os.listdir(curr_dir): 
            if os.path.isfile(os.path.join(curr_dir, file_name)) == False:
                continue
                
            form_id = int(file_name.split('_')[5])
            aes_score = mp_formWise_aesScores[form_id]
            
            image_file_name.append(file_name)
            aesthetic_score.append(aes_score)
            if aes_score >= mean_aesScore:
                aesthetic_quality.append('good')
                prob_good.append(1)
            else:
                aesthetic_quality.append('bad')
                prob_good.append(0)
                
    return pd.DataFrame({'image_file_name': image_file_name,
                       'aesthetic_score': aesthetic_score,
                       'aesthetic_quality': aesthetic_quality,
                       'probability_good': prob_good})

In [25]:
hw_aes_df = prepare_hwAesthetics_df(mp_formWise_aesScores)
hw_aes_df.head()

,image_file_name,aesthetic_score,aesthetic_quality,probability_good
0,02_0002_0_23_1016_1067_1.png,1.333333,bad,0
1,02_0002_0_23_1016_1730_1.png,1.333333,bad,0
2,01_0001_0_15_0916_0422_1.png,3.666667,good,1
3,02_0002_0_24_1016_1542_1.png,2.000000,bad,0
4,01_0001_1_16_0916_0320_1.png,4.000000,good,1


In [26]:
hw_aes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166105 entries, 0 to 166104
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   image_file_name    166105 non-null  object 
 1   aesthetic_score    166105 non-null  float64
 2   aesthetic_quality  166105 non-null  object 
 3   probability_good   166105 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 5.1+ MB


In [29]:
# save prepared dataframe as csv file
hw_aes_df.to_csv(os.path.join('..', 'dataset', 'prepared-datasets', 'aesthetics.csv'), index=False)